In [1]:
!pip install kaggle


/content/sample_data

In [101]:


! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
api_token = {"username":"rishikesh1218","key":"52a10a0aea28c7d07b86c8245c7d55e5"}
import json
import zipfile
import os
with open('/content/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /content/kaggle.json
!kaggle config set -n path -v /content
!kaggle datasets download -d rishikesh1218/hand-gesture-data
os.chdir('/content/hand-gesture-data')
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')

- path is now set to: /content
 98% 442M/452M [00:01<00:00, 312MB/s]
100% 452M/452M [00:01<00:00, 287MB/s]


In [6]:
import zipfile

# Unzip the dataset
local_zip = './hand-gesture-data.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./Hand Gesture Data')
zip_ref.close()

In [63]:
import os

# Directory with our training rock pictures
train_rock_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/train/rock')

# Directory with our training paper pictures
train_paper_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/train/paper')

# Directory with our training scissor pictures
train_scissors_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/train/scissors')

# Directory with validation horse pictures
validation_rock_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/validation/rock')

# Directory with validation human pictures
validation_paper_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/validation/paper')

# Directory with validation horse pictures
validation_scissors_dir = os.path.join('./Hand Gesture Data/Rock-Paper-Scissors/validation/scissors')

In [103]:
import os

path = os.getcwd()

In [102]:
train_rock_names = os.listdir(train_rock_dir)
train_paper_names = os.listdir(train_paper_dir)
train_scissors_names = os.listdir(train_scissors_dir)

print('total training rock images:', len(os.listdir(train_rock_dir)))
print('total training paper images:', len(os.listdir(train_paper_dir)))
print('total training scissor images:', len(os.listdir(train_scissors_dir)))

total training rock images: 840
total training paper images: 840
total training scissor images: 840


In [10]:
import tensorflow as tf

In [110]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training when the accuracy reaches 1

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check the loss
    if(logs.get('accuracy') == 1):

      # Stop if threshold is met
      print("\nAccuracy is equal to 1 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [104]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
# Print the model summary
model.summary()

# Use same settings
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 35, 35, 64)      

In [105]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './Hand Gesture Data/Rock-Paper-Scissors/train/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')


Found 2520 images belonging to 3 classes.


In [106]:
history = model.fit(
      train_generator,
      steps_per_epoch=19,
      epochs=10,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/10
19/19 [==============================] - 14s 579ms/step - loss: 1.8805 - accuracy: 0.5619
Epoch 2/10
19/19 [==============================] - 12s 630ms/step - loss: 0.1405 - accuracy: 0.9646
Epoch 3/10
19/19 [==============================] - 13s 674ms/step - loss: 0.0137 - accuracy: 0.9979
Epoch 4/10
19/19 [==============================] - ETA: 0s - loss: 0.0034 - accuracy: 1.0000
Accuracy is equal to 1 so cancelling training!
19/19 [==============================] - 15s 785ms/step - loss: 0.0034 - accuracy: 1.0000


In [109]:
## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/hand-gesture-data/Hand Gesture Data/Rock-Paper-Scissors/validation/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  if((classes[0][0]>classes[0][1]) and (classes[0][0]>classes[0][2])):
    print("paper")
  elif((classes[0][1]>classes[0][0]) and (classes[0][1]>classes[0][2])):
    print("rock")
  else:
    print("scissor")


Saving paper1.png to paper1.png
Saving paper-hires1.png to paper-hires1.png
Saving rock1.png to rock1.png
Saving rock-hires1.png to rock-hires1.png
Saving scissors1.png to scissors1.png
Saving scissors-hires1.png to scissors-hires1.png
1/1 [==============================] - 0s 76ms/step
paper
1/1 [==============================] - 0s 20ms/step
paper
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 18ms/step
rock
1/1 [==============================] - 0s 20ms/step
rock
1/1 [==============================] - 0s 20ms/step
scissor
